In [78]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os
import re 
from collections import defaultdict
from PIL import Image
import matplotlib.cm as cm

#Here we define the diractory as follows :
Y_Data_Dir = "/home/yazeed/Desktop/Ellipse_Simulation/Ellipes-Y-axis" #This is for Y-Ellipses 
X_Data_Dir = "/home/yazeed/Desktop/Ellipse_Simulation/Ellipse-X-axis" #This is for the X-Ellipses
org_Data_Dir = "/home/yazeed/Desktop/Ellipse_Simulation/Organized_Data"
Y_Data_Path = "/home/yazeed/Desktop/Ellipse_Simulation/Organized_Data/Y-Ellipses"
X_Data_Path = "/home/yazeed/Desktop/Ellipse_Simulation/Organized_Data/X-Ellipses"
Figure_Path = "/home/yazeed/Desktop/Ellipse_Simulation/Organized_Data/Figures"
PROJECT_DIR = [Y_Data_Dir, X_Data_Dir, org_Data_Dir, Y_Data_Path, X_Data_Path, Figure_Path]
for dir in PROJECT_DIR:
    if not os.path.exists(dir):
        print(f"Path does not exsits {dir}. Creating it .....")
        os.mkdir(dir)
        
#Then we get the files name in each folder as follows 
Y_file_name = [f for f in os.listdir(Y_Data_Dir) if f.endswith(".txt")]
X_file_name = [f for f in os.listdir(X_Data_Dir) if f.endswith(".txt")]

#Then we need to sort the files based on the different PMMA thicknesses and energies 
thickness_pattern= re.compile(r'PMMA_(\d+)_mm')
energy_pattern = re.compile(r'Energy_(\d+(_\d+)?)_Mev')

# Create a dictionary to organize files by (thickness, energy)
Y_files_by_thickness_and_energy = defaultdict(list)
X_files_by_thickness_and_energy = defaultdict(list)

# Process each file name within the Y_Data_Ellipses

for file_name in Y_file_name:
    thickness_match = thickness_pattern.search(file_name)
    energy_match = energy_pattern.search(file_name)
    
    if thickness_match:
        thickness = thickness_match.group(1)
    else:
        thickness = '20' 

    if energy_match:
        energy = energy_match.group(1)
    else:
        energy = 'unknown' 
    
    Y_files_by_thickness_and_energy[(thickness, energy)].append(file_name)

for file_name in X_file_name:
    thickness_match = thickness_pattern.search(file_name)
    energy_match = energy_pattern.search(file_name)
    
    if thickness_match:
        thickness = thickness_match.group(1)
    else:
        thickness = '20' 

    if energy_match:
        energy = energy_match.group(1)
    else:
        energy = 'unknown' 
    
    X_files_by_thickness_and_energy[(thickness, energy)].append(file_name)


# Create directories and move files
for (thickness, energy), files in Y_files_by_thickness_and_energy.items():
    # Define the directory path based on thickness and energy
    dir_path = os.path.join(Y_Data_Path, f"PMMA_{thickness}_mm_Energy_{energy}_Mev")
    
    # Create directory if it doesn't exist
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    
    # Move files to the corresponding directory
    for file in files:
        src = os.path.join(Y_Data_Dir, file)
        dst = os.path.join(dir_path, file)
        os.rename(src, dst)
        print(f"Moved file {file} to {dir_path}")


# Create directories and move files
for (thickness, energy), files in X_files_by_thickness_and_energy.items():
    # Define the directory path based on thickness and energy
    dir_path = os.path.join(X_Data_Path, f"PMMA_{thickness}_mm_Energy_{energy}_Mev")
    
    # Create directory if it doesn't exist
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    
    # Move files to the corresponding directory
    for file in files:
        src = os.path.join(X_Data_Dir, file)
        dst = os.path.join(dir_path, file)
        os.rename(src, dst)
        print(f"Moved file {file} to {dir_path}")

print("File organization complete.") #EventID

File organization complete.


In [72]:
def process_files_for_angles(file_names, path, energy):
    for angle in angles:
        Vx = []
        Vy = []
        E_id = []
        # Filter files for the specific angle
        angle_files = [f for f in file_names if angle_pattern.search(f) and int(angle_pattern.search(f).group(1)) == angle]
        
        for file_name in angle_files:
            file_path = os.path.join(path, file_name)
            df = pd.read_csv(file_path, delimiter=" ")
            # Ensure the columns are present
            if 'VertexX' in df.columns and 'VertexY' and 'EventID' in df.columns:
                Vx.extend(df['VertexX'].tolist())
                Vy.extend(df['VertexY'].tolist())
                E_id.extend(df['EventID'].tolist())
                
            else:
                print(f"Columns 'Vertex X' and 'Vertex Y' not found in {file_name}")
        
        # Save the data for the current angle
        if Vx and Vy:  # Ensure there is data to save
            data_df = pd.DataFrame({'VertexX': Vx, 'VertexY': Vy, 'EventID': E_id})
            output_path = os.path.join(path, f'PMMA_14_mm_Energy_{energy}_Angle_{angle}.csv')
            data_df.to_csv(output_path, index=False)
            print(f"Data for Angle {angle} saved to {output_path}")

def load_data_for_angle(path, energy, angle):
    file_path = os.path.join(path, f'PMMA_14_mm_Energy_{energy}_Angle_{angle}.csv')
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        return df
    else:
        print(f"File {file_path} does not exist.")
        return None

def plot_2d_histogram(df, energy, angle, output_path, PMMA_thickness):
    plt.figure(figsize=(8, 6))
    plt.hist2d(df['VertexX'], df['VertexY'], bins=30, cmap='viridis', cmin=1)
    plt.colorbar(label='Counts')
    plt.title(f'2D Histogram of Vertex X vs Vertex Y\n PMMA thickness: {PMMA_thickness} mm, Energy: {energy} MeV, Angle: {angle}°')
    plt.xlabel('Vertex X')
    plt.ylabel('Vertex Y')
    plt.grid(True)
    plt.savefig(output_path)
    plt.close()

def create_histograms(Path_PMMA_Energy_1_5, Path_PMMA_Energy_1, Path_PMMA_Energy_2, Histogram_Output_Dir, angles, PMMA_thickness):
    for energy, path in zip(['1_5_Mev', '1_Mev', '2_Mev'], 
                            [Path_PMMA_Energy_1_5, Path_PMMA_Energy_1, Path_PMMA_Energy_2]):
        for angle in angles:
            df = load_data_for_angle(path, energy, angle)
            if df is not None:
                # Create a unique directory for each energy
                energy_dir = os.path.join(Histogram_Output_Dir, f'Energy_{energy}')
                if not os.path.exists(energy_dir):
                    os.makedirs(energy_dir)
                
                output_path = os.path.join(energy_dir, f'2D_Histogram_Angle_{angle}.png')
                plot_2d_histogram(df, energy, angle, output_path, PMMA_thickness)
                print(f"Histogram saved to {output_path}")

In [74]:
# Define paths to the directories
Path_PMMA_14_Energy_1_5 = "/home/yazeed/Desktop/Ellipse_Simulation/Organized_Data/Y-Ellipses/PMMA_14_mm_Energy_1_5_Mev"
Path_PMMA_14_Energy_1 = "/home/yazeed/Desktop/Ellipse_Simulation/Organized_Data/Y-Ellipses/PMMA_14_mm_Energy_1_Mev"
Path_PMMA_14_Energy_2 = "/home/yazeed/Desktop/Ellipse_Simulation/Organized_Data/Y-Ellipses/PMMA_14_mm_Energy_2_Mev"

# Define the angles of interest
angles = [0, 5, 10, 15, 20, 25]

# Define regex pattern to extract angles
angle_pattern = re.compile(r'angle_(\d+)_degree')

# Process files for each energy
def process_all_energies():
    # Process for Energy 1.5 Mev
    file_names_PMMA_14_Energy_1_5 = [f for f in os.listdir(Path_PMMA_14_Energy_1_5) if f.endswith(".txt")]
    process_files_for_angles(file_names_PMMA_14_Energy_1_5, Path_PMMA_14_Energy_1_5, '1_5_Mev')
    
    # Process for Energy 1 Mev
    file_names_PMMA_14_Energy_1 = [f for f in os.listdir(Path_PMMA_14_Energy_1) if f.endswith(".txt")]
    process_files_for_angles(file_names_PMMA_14_Energy_1, Path_PMMA_14_Energy_1, '1_Mev')
    
    # Process for Energy 2 Mev
    file_names_PMMA_14_Energy_2 = [f for f in os.listdir(Path_PMMA_14_Energy_2) if f.endswith(".txt")]
    process_files_for_angles(file_names_PMMA_14_Energy_2, Path_PMMA_14_Energy_2, '2_Mev')

process_all_energies()

print("Data extraction and saving complete.")

Data for Angle 0 saved to /home/yazeed/Desktop/Ellipse_Simulation/Organized_Data/Y-Ellipses/PMMA_14_mm_Energy_1_5_Mev/PMMA_14_mm_Energy_1_5_Mev_Angle_0.csv
Data for Angle 5 saved to /home/yazeed/Desktop/Ellipse_Simulation/Organized_Data/Y-Ellipses/PMMA_14_mm_Energy_1_5_Mev/PMMA_14_mm_Energy_1_5_Mev_Angle_5.csv
Data for Angle 10 saved to /home/yazeed/Desktop/Ellipse_Simulation/Organized_Data/Y-Ellipses/PMMA_14_mm_Energy_1_5_Mev/PMMA_14_mm_Energy_1_5_Mev_Angle_10.csv
Data for Angle 15 saved to /home/yazeed/Desktop/Ellipse_Simulation/Organized_Data/Y-Ellipses/PMMA_14_mm_Energy_1_5_Mev/PMMA_14_mm_Energy_1_5_Mev_Angle_15.csv
Data for Angle 20 saved to /home/yazeed/Desktop/Ellipse_Simulation/Organized_Data/Y-Ellipses/PMMA_14_mm_Energy_1_5_Mev/PMMA_14_mm_Energy_1_5_Mev_Angle_20.csv
Data for Angle 25 saved to /home/yazeed/Desktop/Ellipse_Simulation/Organized_Data/Y-Ellipses/PMMA_14_mm_Energy_1_5_Mev/PMMA_14_mm_Energy_1_5_Mev_Angle_25.csv
Data for Angle 0 saved to /home/yazeed/Desktop/Ellipse_S

In [96]:
Path_PMMA_14_Energy_1_5 = "/home/yazeed/Desktop/Ellipse_Simulation/Organized_Data/X-Ellipses/PMMA_14_mm_Energy_1_5_Mev"
Path_PMMA_14_Energy_1 = "/home/yazeed/Desktop/Ellipse_Simulation/Organized_Data/X-Ellipses/PMMA_14_mm_Energy_1_Mev"
Path_PMMA_14_Energy_2 = "/home/yazeed/Desktop/Ellipse_Simulation/Organized_Data/X-Ellipses/PMMA_14_mm_Energy_2_Mev"


PMMA_PATHS = [Path_PMMA_14_Energy_1_5, Path_PMMA_14_Energy_1, Path_PMMA_14_Energy_2]

for Data_Path in PMMA_PATHS:
    txt_files = [f for f in os.listdir(Data_Path) if f.endswith('.txt')]

    for fName in txt_files:
        Data = pd.read_csv(os.path.join(Data_Path, fName), delimiter=" ")

        grouped = Data.groupby('EventID')

        for event_id, group in grouped:
            Vx_CP = group['VertexX']
            Vy_CP = group['VertexY']

            Vx_CP_new = Vx_CP - np.min(Vx_CP)
            Vy_CP_new = Vy_CP - np.min(Vy_CP)

            hist, xedges, yedges = np.histogram2d(Vx_CP_new, Vy_CP_new, bins=(20, 20), range=[[0, 40], [0, 40]])

            hist_norm = hist / hist.max()

            cmap = cm.get_cmap('viridis')  
            hist_colored = cmap(hist_norm)

            hist_img = Image.fromarray((hist_colored[:, :, :3] * 255).astype(np.uint8))

            figure = os.path.join(Figure_Path, 'X_Ellipse_PMMA_14_mm')
            if not os.path.exists(figure):
                os.makedirs(figure)

            output_file = os.path.join(figure, f"{os.path.splitext(fName)[0]}_EventID_{event_id}.png")

            hist_img.save(output_file)

        print(f"Processed file {fName} and saved colored histograms for all EventIDs.")

print("All files have been processed.")

/tmp/ipykernel_10311/3845790869.py:27: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap('viridis')


Processed file Ellipes_angle_10_degree_pos_X_0_Y_0_Z_20_Energy_1_5_Mev_PMMA_14_mm_20240827_222212.txt and saved colored histograms for all EventIDs.


/tmp/ipykernel_10311/3845790869.py:27: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap('viridis')


Processed file Ellipes_angle_5_degree_pos_X_0_Y_0_Z_20_Energy_1_5_Mev_PMMA_14_mm_20240827_222211.txt and saved colored histograms for all EventIDs.


/tmp/ipykernel_10311/3845790869.py:27: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap('viridis')


...

Processed file Ellipes_angle_25_degree_pos_X_20_Y_0_Z_20_Energy_2_Mev_PMMA_14_mm_20240827_222212.txt and saved colored histograms for all EventIDs.


/tmp/ipykernel_10311/3845790869.py:27: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap('viridis')


Processed file Ellipes_angle_15_degree_pos_X_10_Y_0_Z_20_Energy_2_Mev_PMMA_14_mm_20240827_222211.txt and saved colored histograms for all EventIDs.


/tmp/ipykernel_10311/3845790869.py:27: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap('viridis')


Processed file Ellipes_angle_35_degree_pos_X_20_Y_0_Z_20_Energy_2_Mev_PMMA_14_mm_20240827_222213.txt and saved colored histograms for all EventIDs.
All files have been processed.
